In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt

import gym

from gym import spaces

import players
import envs
from evaluation import generate_history, is_t4t

/home/peter/anaconda3/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
import ray
from ray import tune
from ray.rllib.examples.env.multi_agent import MultiAgentCartPole
from ray.rllib.examples.models.shared_weights_model import \
    SharedWeightsModel1, SharedWeightsModel2, TF2SharedWeightsModel, \
    TorchSharedWeightsModel
from ray.rllib.models import ModelCatalog
# from ray.rllib.policy import PolicySpec
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.tune.registry import register_env

from ray.rllib.env.multi_agent_env import MultiAgentEnv


import ray
from ray.rllib.agents.ppo import PPOTrainer, PPOTorchPolicy, PPOTFPolicy
from ray.rllib.agents.ppo import DEFAULT_CONFIG as DEFAULT_CONFIG_PPO

from ray.rllib.agents.dqn import DQNTrainer, DQNTorchPolicy, DQNTFPolicy
from ray.rllib.agents.dqn import  DEFAULT_CONFIG as DEFAULT_CONFIG_DQN


from ray.tune.registry import register_env
from ray.tune.logger import pretty_print

from ray.rllib.agents.callbacks import DefaultCallbacks
from ray.rllib.env import BaseEnv
from ray.rllib.evaluation import MultiAgentEpisode, RolloutWorker
from ray.rllib.policy import Policy
from ray.rllib.policy.sample_batch import SampleBatch

ray.init(ignore_reinit_error=True, log_to_driver=False)

2021-08-21 11:12:37,510	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.21',
 'raylet_ip_address': '192.168.1.21',
 'redis_address': '192.168.1.21:6379',
 'object_store_address': '/tmp/ray/session_2021-08-21_11-12-35_603149_139304/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-08-21_11-12-35_603149_139304/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-08-21_11-12-35_603149_139304',
 'metrics_export_port': 61335,
 'node_id': '95efe6761ce1922a4428ee7acd11dffa3d4d05bd385dde22cd3b4b07'}

In [3]:
register_env('two_agent_MG_env', lambda c: envs.TwoAgentMatrixGameEnv(c))

register_env('two_agent_t4t_MG_env', lambda c: envs.TwoAgentSeparateMatrixGameEnv(c))

register_env('MG_t4tTD_env', lambda c: envs.MatrixGameEnv(player2=players.TitForTatThenDefect()))
# register_env('MG_t4t_env', lambda c: envs.MatrixGameEnv(player2=players.TitForTat()))
register_env('MG_t4t_env', lambda c: envs.MatrixGameEnv())

In [4]:
trainer_config_dqn0 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn0['num_workers'] = 3
trainer_config_dqn0['n_step'] = 3
trainer_config_dqn0['noisy'] = True
trainer_config_dqn0['v_min'] = -10.
trainer_config_dqn0['v_max'] = 10.
trainer_config_dqn0['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn0['framework'] = 'torch'
# trainer_config_dqn0['framework'] = 'tf'


In [5]:
trainer_config_dqn1 = DEFAULT_CONFIG_DQN.copy()
# trainer_config_dqn1['num_workers'] = 3
trainer_config_dqn1['n_step'] = 3
trainer_config_dqn1['noisy'] = True
trainer_config_dqn1['v_min'] = -10.
trainer_config_dqn1['v_max'] = 10.
trainer_config_dqn1['model']['fcnet_hiddens'] = [256,32,8]
trainer_config_dqn1['framework'] = 'torch'
# trainer_config_dqn1['framework'] = 'tf'


In [6]:
# set up stuff for pre-training with tit-for-tat

def env_creator(_):
    return envs.TwoAgentSeparateMatrixGameEnv()
single_env = envs.TwoAgentSeparateMatrixGameEnv()
env_name = 'two_agent_t4t_MG_env'
register_env(env_name, env_creator)



obs_space = single_env.observation_space
act_space = single_env.action_space
num_agents = single_env.num_agents
# DQNTorchPolicy0 = DQNTorchPolicy.copy()

def gen_policy(i):
#     return (PPOTorchPolicy, obs_space, act_space, {})
    if i == 0:
        conf = trainer_config_dqn0
#         conf['restore']='wtf_this better not work'#results.get_last_checkpoint()
#         return (DQNTorchPolicy0, obs_space, act_space, conf)
#         policy0 = copy.copy(agent0.get_policy())
#         policy0 = DQNTorchPolicy(obs_space, act_space, conf)
        policy0 = DQNTorchPolicy
#         policy0 = agent0.get_policy
#         policy0.set_weights(agent0.get_weights()['default_policy'])
#         policy0 = lambda c: policy0
#         return policy0
        return (policy0, obs_space, act_space, conf)

    elif i == 1:
        conf = trainer_config_dqn1
        return (DQNTorchPolicy, obs_space, act_space, conf)
    else:
        print('NO CONF!')
    
    return (DQNTorchPolicy, obs_space, act_space, conf)

policy_graphs = {}

# def restore_fn(agent_id):
# #     if agent_id == 0:
#     return results.get_last_checkpoint()
# #     else:
# #         return None
    
# restore_dict = {}
# for i in range(num_agents):
#     restore_dict['agent-' + str(i)] = restore_fn(i)

for i in range(num_agents):
    policy_graphs['agent-' + str(i)] = gen_policy(i)
def policy_mapping_fn(agent_id):
        return 'agent-' + str(agent_id)

In [49]:

class MyCallbacks(DefaultCallbacks):
    def on_episode_start(self, *, worker: RolloutWorker, base_env: BaseEnv,
                         policies: Dict[str, Policy],
                         episode: MultiAgentEpisode, env_index: int, **kwargs):
        # Make sure this episode has just been started (only initial obs
        # logged so far).
        assert episode.length == 0, 
        episode.user_data["it"] = []
        episode.hist_data["it"] = []

#     def on_episode_step(self, *, worker: RolloutWorker, base_env: BaseEnv,
#                         episode: MultiAgentEpisode, env_index: int, **kwargs):
#         # Make sure this episode is ongoing.
#         assert episode.length > 0, \
#             "ERROR: `on_episode_step()` callback should not be called right " \
#             "after env reset!"
#         pole_angle = abs(episode.last_observation_for()[2])
#         raw_angle = abs(episode.last_raw_obs_for()[2])
#         assert pole_angle == raw_angle
#         episode.user_data["pole_angles"].append(pole_angle)

    def on_episode_end(self, *, worker: RolloutWorker, base_env: BaseEnv,
                       policies: Dict[str, Policy], episode: MultiAgentEpisode,
                       env_index: int, **kwargs):
        # Make sure this episode is really done.
        assert episode.batch_builder.policy_collectors[
            "default_policy"].buffers["dones"][-1], \
            "ERROR: `on_episode_end()` should only be called " \
            "after episode is done!"
        pole_angle = np.mean(episode.user_data["pole_angles"])
        print("episode {} (env-idx={}) ended with length {} and pole "
              "angles {}".format(episode.episode_id, env_index, episode.length,
                                 pole_angle))
        episode.custom_metrics["pole_angle"] = pole_angle
        episode.hist_data["pole_angles"] = episode.user_data["pole_angles"]

    def on_sample_end(self, *, worker: RolloutWorker, samples: SampleBatch,
                      **kwargs):
        print("returned sample batch of size {}".format(samples.count))

    def on_train_result(self, *, trainer, result: dict, **kwargs):
        print("trainer.train() result: {} -> {} episodes".format(
            trainer, result["episodes_this_iter"]))
        # you can mutate the result dict to add new fields to return
        result["callback_ok"] = True

    def on_learn_on_batch(self, *, policy: Policy, train_batch: SampleBatch,
                          result: dict, **kwargs) -> None:
        result["sum_actions_in_train_batch"] = np.sum(train_batch["actions"])
        print("policy.learn_on_batch() result: {} -> sum actions: {}".format(
            policy, result["sum_actions_in_train_batch"]))

    def on_postprocess_trajectory(
            self, *, worker: RolloutWorker, episode: MultiAgentEpisode,
            agent_id: str, policy_id: str, policies: Dict[str, Policy],
            postprocessed_batch: SampleBatch,
            original_batches: Dict[str, SampleBatch], **kwargs):
        print("postprocessed {} steps".format(postprocessed_batch.count))
        if "num_batches" not in episode.custom_metrics:
            episode.custom_metrics["num_batches"] = 0
        episode.custom_metrics["num_batches"] += 1


SyntaxError: invalid syntax (<ipython-input-49-c34c07422145>, line 7)

In [7]:

class MyCallbacks(DefaultCallbacks):

    def on_train_result(self, *, trainer, result: dict, **kwargs) -> None:
        """Called at the end of Trainable.train().

        Args:
            trainer (Trainer): Current trainer instance.
            result (dict): Dict of results returned from trainer.train() call.
                You can mutate this object to add additional metrics.
            kwargs: Forward compatibility placeholder.
        """

        if self.legacy_callbacks.get("on_train_result"):
            self.legacy_callbacks["on_train_result"]({
                "trainer": trainer,
                "result": result,
            })
        result['t4t_frac_agent-0'], result['coop_frac_agent-0'] = is_t4t(trainer,n_samples=3, policy_id='agent-0')
        result['t4t_frac_agent-1'], result['coop_frac_agent-1'] = is_t4t(trainer,n_samples=3, policy_id='agent-1')

        

In [15]:
trainer = DQNTrainer(config=config)

2021-08-21 11:14:18,953	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [ ]:
trainer.re

In [16]:
trainer.compute_action(np.random.rand(400), 'policy_id')

2021-08-21 11:14:38,483	WARNING deprecation.py:33 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


KeyError: 'default_policy'

In [1]:
# make config for pretraining with tit-for-tat

config={
    "log_level": "WARN",
#     "num_workers": 3,
#     "num_cpus_for_driver": 1,
#     "num_cpus_per_worker": 1,
#     "lr": 5e-3,
#     "model":{"fcnet_hiddens": [1024, 512,256,32,8]},
    "multiagent": {
        "policies": policy_graphs,
        "policy_mapping_fn": policy_mapping_fn,
    },
    "env": "two_agent_t4t_MG_env",
    'framework': 'torch',
    "callbacks": MyCallbacks,
#     'framework': 'tf',
#             "resume":True,
#         "restore": results.get_last_checkpoint()
    
}

NameError: name 'policy_graphs' is not defined

In [11]:
# pretraining against tit-for-tat

exp_name = 'TA_TEST3'
exp_dict = {
        'name': exp_name,
        'run_or_experiment': 'DQN',
        "stop": {
            "training_iteration": 5
        },
        'checkpoint_freq': 0,
        'checkpoint_at_end': True,
        "config": config,
}
# ray.init()
MA_result=tune.run(**exp_dict)

Trial name,status,loc
DQN_two_agent_t4t_MG_env_24a63_00000,PENDING,


Result for DQN_two_agent_t4t_MG_env_24a63_00000:
  agent_timesteps_total: 2000
  coop_frac_agent-0: 1.0
  coop_frac_agent-1: 0.33333333333333337
  custom_metrics: {}
  date: 2021-08-21_11-13-00
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 461.0
  episode_reward_mean: 446.5
  episode_reward_min: 433.0
  episodes_this_iter: 10
  episodes_total: 10
  experiment_id: 70210c90d0e84263b7b3c510bb85bdef
  hostname: coolo-computer
  info:
    last_target_update_ts: 1000
    learner:
      agent-0:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_lr: 0.0005
          grad_gnorm: 6.6504621505737305
          max_q: 0.8930140733718872
          mean_q: 0.40550529956817627
          mean_td_error: -4.355594635009766
          min_q: -0.1307121366262436
        model: {}
        td_error: "[-4.3638225  -6.823642   -4.6358457   0.15035672 -2.7031853  -0.4177984\n\
          \ -4.919959   -8.91489    -2.1450436  -7.801

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_24a63_00000,RUNNING,192.168.1.21:139440,1,3.12891,1000,446.5,461,433,100


Result for DQN_two_agent_t4t_MG_env_24a63_00000:
  agent_timesteps_total: 4000
  coop_frac_agent-0: 1.0
  coop_frac_agent-1: 1.0
  custom_metrics: {}
  date: 2021-08-21_11-13-08
  done: false
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 513.0
  episode_reward_mean: 458.65
  episode_reward_min: 433.0
  episodes_this_iter: 10
  episodes_total: 20
  experiment_id: 70210c90d0e84263b7b3c510bb85bdef
  hostname: coolo-computer
  info:
    last_target_update_ts: 1504
    learner:
      agent-0:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_lr: 0.0005
          grad_gnorm: 1.8771191835403442
          max_q: 11.89836311340332
          mean_q: 7.8602705001831055
          mean_td_error: 0.35999757051467896
          min_q: 3.2044410705566406
        model: {}
        td_error: "[ 1.8426661   0.07402658 -5.0726495  -1.833005   -0.6509576   2.0006084\n\
          \  0.29657173  0.2269206   0.42355442 -3.276247   -2.8512645 

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_24a63_00000,RUNNING,192.168.1.21:139440,2,11.2764,2000,458.65,513,433,100


...

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_24a63_00000,RUNNING,192.168.1.21:139440,4,26.8587,4000,474.3,513,433,100


Result for DQN_two_agent_t4t_MG_env_24a63_00000:
  agent_timesteps_total: 10000
  coop_frac_agent-0: 1.0
  coop_frac_agent-1: 0.33333333333333337
  custom_metrics: {}
  date: 2021-08-21_11-13-32
  done: true
  episode_len_mean: 100.0
  episode_media: {}
  episode_reward_max: 531.0
  episode_reward_mean: 481.9
  episode_reward_min: 433.0
  episodes_this_iter: 10
  episodes_total: 50
  experiment_id: 70210c90d0e84263b7b3c510bb85bdef
  hostname: coolo-computer
  info:
    last_target_update_ts: 4528
    learner:
      agent-0:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_lr: 0.0005
          grad_gnorm: 1.7683415412902832
          max_q: 40.280521392822266
          mean_q: 26.401500701904297
          mean_td_error: 1.9872442483901978
          min_q: 3.664611339569092
        model: {}
        td_error: "[ 3.6835976  10.354925    2.4421482  -9.174738   -1.6979523   3.4056368\n\
          \  0.55655766  3.277443   17.560732    1.688365

Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_24a63_00000,RUNNING,192.168.1.21:139440,5,34.651,5000,481.9,531,433,100


Trial name,status,loc,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
DQN_two_agent_t4t_MG_env_24a63_00000,TERMINATED,,5,34.651,5000,481.9,531,433,100


2021-08-21 11:13:32,529	INFO tune.py:550 -- Total run time: 40.66 seconds (40.37 seconds for the tuning loop).


In [12]:
trainer_config_dqn0

{'num_workers': 0,
 'num_envs_per_worker': 1,
 'create_env_on_driver': False,
 'rollout_fragment_length': 4,
 'batch_mode': 'truncate_episodes',
 'gamma': 0.99,
 'lr': 0.0005,
 'train_batch_size': 32,
 'model': {'_use_default_native_models': False,
  'fcnet_hiddens': [256, 32, 8],
  'fcnet_activation': 'tanh',
  'conv_filters': None,
  'conv_activation': 'relu',
  'post_fcnet_hiddens': [],
  'post_fcnet_activation': 'relu',
  'free_log_std': False,
  'no_final_linear': False,
  'vf_share_layers': True,
  'use_lstm': False,
  'max_seq_len': 20,
  'lstm_cell_size': 256,
  'lstm_use_prev_action': False,
  'lstm_use_prev_reward': False,
  '_time_major': False,
  'use_attention': False,
  'attention_num_transformer_units': 1,
  'attention_dim': 64,
  'attention_num_heads': 1,
  'attention_head_dim': 32,
  'attention_memory_inference': 50,
  'attention_memory_training': 50,
  'attention_position_wise_mlp_dim': 32,
  'attention_init_gru_gate_bias': 2.0,
  'attention_use_n_prev_actions': 0,
  

In [17]:
agent0 = DQNTrainer(config=config)

2021-08-21 11:14:57,672	WARNING util.py:55 -- Install gputil for GPU system monitoring.


In [37]:
p0=MA_result.get_best_trial(metric="episode_reward_mean", mode="max").config['multiagent']['policies']['agent-0']

In [45]:
t1 = MA_result.get_best_trial(metric="episode_reward_mean", mode="max")

In [62]:
t1.config['multiagent']


{'policies': {'agent-0': (ray.rllib.policy.policy_template.DQNTorchPolicy,
   Box(0.0, 1.0, (400,), float32),
   Discrete(2),
   {'num_workers': 0,
    'num_envs_per_worker': 1,
    'create_env_on_driver': False,
    'rollout_fragment_length': 4,
    'batch_mode': 'truncate_episodes',
    'gamma': 0.99,
    'lr': 0.0005,
    'train_batch_size': 32,
    'model': {'_use_default_native_models': False,
     'fcnet_hiddens': [256, 32, 8],
     'fcnet_activation': 'tanh',
     'conv_filters': None,
     'conv_activation': 'relu',
     'post_fcnet_hiddens': [],
     'post_fcnet_activation': 'relu',
     'free_log_std': False,
     'no_final_linear': False,
     'vf_share_layers': True,
     'use_lstm': False,
     'max_seq_len': 20,
     'lstm_cell_size': 256,
     'lstm_use_prev_action': False,
     'lstm_use_prev_reward': False,
     '_time_major': False,
     'use_attention': False,
     'attention_num_transformer_units': 1,
     'attention_dim': 64,
     'attention_num_heads': 1,
     'at

In [18]:
lc = MA_result.get_last_checkpoint(metric="episode_reward_mean", mode="max")

In [19]:
lc

'/home/peter/ray_results/TA_TEST3/DQN_two_agent_t4t_MG_env_24a63_00000_0_2021-08-21_11-12-52/checkpoint_000005/checkpoint-5'

In [21]:
agent0.restore(lc)

2021-08-21 11:15:13,870	INFO trainable.py:382 -- Restored on 192.168.1.21 from checkpoint: /home/peter/ray_results/TA_TEST3/DQN_two_agent_t4t_MG_env_24a63_00000_0_2021-08-21_11-12-52/checkpoint_000005/checkpoint-5
2021-08-21 11:15:13,871	INFO trainable.py:390 -- Current state after restoring: {'_iteration': 5, '_timesteps_total': None, '_time_total': 34.65098333358765, '_episodes_total': 50}


In [23]:
agent0.compute_action(np.random.randn(400), policy_id='agent-0')

1